In [1]:
# Imports

import plotly
import optuna
import mlflow
import dagshub
import mlflow.sklearn

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
# Setting up DagsHub

dagshub.init(repo_owner='SushrutGaikwad', repo_name='youtube-comments-analyzer', mlflow=True)

Accessing as SushrutGaikwad

Initialized MLflow to track repo "SushrutGaikwad/youtube-comments-analyzer"

Repository SushrutGaikwad/youtube-comments-analyzer initialized!

# Data

In [3]:
PREPROCESSED_DATA_PATH = "../data/processed/reddit_preprocessed.csv"
df = pd.read_csv(PREPROCESSED_DATA_PATH)
df.dropna(subset=["clean_comment"], inplace=True)
df.shape

(36662, 2)

# Running the experiment

In [4]:
# Setting experiment name

mlflow.set_experiment("Exp 5: Detailed hyperparameter tuning")

<Experiment: artifact_location='mlflow-artifacts:/50f3ef58d8344cd3bd0def333af39f2d', creation_time=1749058038347, experiment_id='6', last_update_time=1749064965442, lifecycle_stage='active', name='Exp 5: Detailed hyperparameter tuning', tags={}>

## Preprocessing

In [5]:
# Remapping class labels from {-1, 0, 1} to {2, 0, 1}
mapping = {
    -1: 2,
    0: 0,
    1: 1
}
df["category"] = df["category"].map(mapping)

# Removing missing sentiments
df.dropna(subset=["category"], inplace=True)

# Feature engineering
ngram_range = (1, 2)
max_features = 1000
vectorizer = CountVectorizer(
    ngram_range=ngram_range,
    max_features=max_features
)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    df["clean_comment"],
    df["category"],
    test_size=0.2,
    random_state=42,
    stratify=df["category"]
)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Undersampling
rus = RandomUnderSampler(random_state=42)
X_train_vectorized, y_train = rus.fit_resample(
    X_train_vectorized,
    y_train
)

## Running the experiment

In [6]:
# Function to log results to MLFlow
def log_to_mlflow(
    model_name,
    model,
    X_train,
    X_test,
    y_train,
    y_test,
    params,
    trial_number
):
    with mlflow.start_run():
        # Tags
        mlflow.set_tag(
            "mlflow.runName", f"Trial_{trial_number}_{model_name}_undersampling_BoW_1000_bigrams"
        )
        mlflow.set_tag("experiment_type", "ML_models_comparison")
        
        # Logging model name as a parameter
        mlflow.log_param("model_name", model_name)
        
        # Logging hyperparameters
        for key, value in params.items():
            mlflow.log_param(key, value)
        
        # Initializing and training the model
        model.fit(X_train, y_train)
        
        # Making predictions on the test set and logging metrics
        y_pred = model.predict(X_test)
        
        # Logging accuracy
        accuracy = accuracy_score(
            y_true=y_test,
            y_pred=y_pred
        )
        mlflow.log_metric("accuracy", accuracy)
        
        # Logging classification report metrics
        classification_rep = classification_report(
            y_true=y_test,
            y_pred=y_pred,
            output_dict=True
        )
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}: {metric}", value)
        
        # Logging the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")
        
        return accuracy

In [7]:
def objective(trial):
    # Hyperparameter space
    C = trial.suggest_float("C", 1e-2, 100.0, log=True)
    kernel = trial.suggest_categorical(
        "kernel", ["linear", "rbf", "poly", "sigmoid"]
    )
    gamma = trial.suggest_categorical(
        "gamma", ["scale", "auto"]
    )
    if kernel == "poly":
        degree = trial.suggest_int("degree", 2, 5)
    else:
        degree = 3
    
    params = {
        "C": C,
        "kernel": kernel,
        "gamma": gamma,
        "degree": degree
    }
    
    model = SVC(
        C=C,
        kernel=kernel,
        gamma=gamma,
        degree=degree,
        random_state=42
    )
    
    # Logging each trial as a separate run
    accuracy = log_to_mlflow(
        model_name="SVM",
        model=model,
        X_train=X_train_vectorized,
        X_test=X_test_vectorized,
        y_train=y_train,
        y_test=y_test,
        params=params,
        trial_number=trial.number
    )
    
    return accuracy

In [8]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

[I 2025-06-05 03:51:28,324] A new study created in memory with name: no-name-780ba638-463a-4b55-90c5-669447586cd2
2025/06/05 03:52:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Trial_0_SVM_undersampling_BoW_1000_bigrams at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6/runs/e43e4974fb50470388aff2549b43de3c
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6


[I 2025-06-05 03:52:24,661] Trial 0 finished with value: 0.6787126687576708 and parameters: {'C': 0.010387072018315745, 'kernel': 'linear', 'gamma': 'scale'}. Best is trial 0 with value: 0.6787126687576708.
2025/06/05 03:53:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Trial_1_SVM_undersampling_BoW_1000_bigrams at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6/runs/81a4003be5ec4fd9ac64d8e4b47de83a
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6


[I 2025-06-05 03:53:26,306] Trial 1 finished with value: 0.6789854084276558 and parameters: {'C': 5.3874117297405135, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 1 with value: 0.6789854084276558.
2025/06/05 04:01:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

[I 2025-06-05 05:10:29,035] Trial 47 finished with value: 0.7612164189281331 and parameters: {'C': 32.696494004189404, 'kernel': 'rbf', 'gamma': 'scale'}. Best is trial 31 with value: 0.779762716487113.
2025/06/05 05:11:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Trial_48_SVM_undersampling_BoW_1000_bigrams at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6/runs/2ac33c27b3c14b8ea5dd6491358ef0ce
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6


[I 2025-06-05 05:11:32,131] Trial 48 finished with value: 0.3915177962634665 and parameters: {'C': 0.8487968884362158, 'kernel': 'poly', 'gamma': 'scale', 'degree': 4}. Best is trial 31 with value: 0.779762716487113.
2025/06/05 05:12:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Trial_49_SVM_undersampling_BoW_1000_bigrams at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6/runs/3549b0e8c7b643479ae6d0aede0f210c
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6


[I 2025-06-05 05:12:31,253] Trial 49 finished with value: 0.5100231828719487 and parameters: {'C': 0.03974167545259752, 'kernel': 'sigmoid', 'gamma': 'scale'}. Best is trial 31 with value: 0.779762716487113.


In [11]:
# Getting the best parameters
best_params = study.best_params
best_model = SVC(
    C=best_params["C"],
    kernel=best_params["kernel"],
    gamma=best_params["gamma"],
    # degree=best_params["degree"],
    random_state=42
)

# Logging the best model
log_to_mlflow(
    model_name="SVM",
    model=best_model,
    X_train=X_train_vectorized,
    X_test=X_test_vectorized,
    y_train=y_train,
    y_test=y_test,
    params=best_params,
    trial_number="best"
)

2025/06/05 05:14:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Trial_best_SVM_undersampling_BoW_1000_bigrams at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6/runs/01082bec725b4e9c98df3a1cfbcc2c22
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/6


0.779762716487113

In [12]:
# Plotting parameter importance
optuna.visualization.plot_param_importances(study).show()

# Plotting optimization history
optuna.visualization.plot_optimization_history(study).show()